<a href="https://colab.research.google.com/github/grommy/Introduction-to-TensorFlow/blob/master/week_4/Exercise4_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [2]:
import tensorflow as tf
import os
import zipfile


!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

--2019-07-02 12:41:41--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2019-07-02 12:41:41 (171 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [0]:
DESIRED_ACCURACY = 0.999

class MetricAchiever(tf.keras.callbacks.Callback):
  def __init__(self, metric='acc', desired_value=DESIRED_ACCURACY):
    self.metric_to_monitor = metric
    self.desired_value = desired_value
    super(MetricAchiever, self).__init__()
  
  def on_epoch_end(self, epoch, logs={}):
    metric = logs.get(self.metric_to_monitor)
    if metric >= self.desired_value:
      print("/n {} value is more than {:.2f}. Stopping training".format(metric, epoch))
      self.model.stop_training = True

metric_achiever_callback = MetricAchiever()

In [26]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(16, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'],
             )

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 61, 61, 16)        4624      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 30, 30, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 14400)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               7373312   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [27]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s',
    target_size=(128, 128),
    batch_size=20,
    class_mode='binary',
    seed=42,
)
      

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [28]:
# This code block should call model.fit_generator and train for
# a number of epochs. 
history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=4,
    epochs=20,
    verbose=1,
    callbacks=[metric_achiever_callback]
)
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/20
4/4 [==============================] - 2s 562ms/step - loss: 327.2466 - acc: 0.5625
Epoch 2/20
4/4 [==============================] - 2s 377ms/step - loss: 2.7688 - acc: 0.4875
Epoch 3/20
4/4 [==============================] - 2s 377ms/step - loss: 0.4430 - acc: 0.8125
Epoch 4/20
4/4 [==============================] - 1s 374ms/step - loss: 0.3353 - acc: 0.8250
Epoch 5/20
4/4 [==============================] - 2s 377ms/step - loss: 0.1624 - acc: 0.9375
Epoch 6/20
4/4 [==============================] - 2s 377ms/step - loss: 0.1264 - acc: 0.9750
Epoch 7/20
4/4 [==============================] - 2s 376ms/step - loss: 0.0898 - acc: 1.0000


In [0]:
x, y = train_generator.next()

In [16]:
print(x.shape)

(20, 128, 128, 3)


In [19]:
print(y.shape)

(20,)
